# Generate a TLDR of Google search results on Trump cabinet picks

Learn how to use Exfunc and OpenAI to generate a TLDR of Google search results

## Getting Started

### Install packages

In [ ]:
!pip install exfunc openai

### Configure API keys

You will need to provide API keys. You can get your Exfunc API key [here](https://exfunc.com) and your OpenAI API key [here](https://www.openai.com/).

Ensure both API keys are accessible in your local environment.


In [ ]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API key:\n")
if "EXFUNC_API_KEY" not in os.environ:
    os.environ["EXFUNC_API_KEY"] = getpass.getpass("Exfunc API key:\n")

### Configure clients

In [ ]:
from exfunc import Exfunc
from openai import OpenAI

exfunc = Exfunc()
openai = OpenAI()

## Generating a TLDR of Google search results

### Search web

In [ ]:
search_web_response = exfunc.google.search_web(request={
    "query": "trump cabinet picks",
    "count": 10,
})
urls = [item.url for item in search_web_response.results]

### Get articles in markdown

In [ ]:
from concurrent.futures import ThreadPoolExecutor

def get_markdown(url):
    max_len = 10000
    try:
        response = exfunc.navigator.scrape(request={"url": url})
        return response.markdown[:max_len]
    except:
        return None

markdowns = []
with ThreadPoolExecutor(max_workers=min(10, len(urls))) as executor:
    for markdown in executor.map(get_markdown, urls):
        if markdown:
            markdowns.append(markdown)

### Generate TLDR

In [ ]:
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "TLDR in bullet points."},
        {"role": "user", "content": "\n\n".join(markdowns)}
    ],
    temperature=0
)

completion = response.choices[0].message.content.strip()

In [ ]:
from IPython.display import display, Markdown

display(Markdown(completion))